In [ ]:
### FIG 14 ###
# offset = Uniform(-1, 1) * Range(90, 180) deg

# ego = Car on drivableRoad,
#         facing offset relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# otherCar = Car on visible road,
#             facing Range(-15, 15) deg relative to roadDirection

# require (distance from ego to otherCar) < 10

In [ ]:
%cd ..

In [ ]:
import json
import os
import pickle

from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.stages.in_view import InView
from optimized_ingestion.stages.decode_frame.parallel_decode_frame import ParallelDecodeFrame
from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.filter_car_facing_sideway import FilterCarFacingSideway
from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT
# from optimized_ingestion.stages.tracking_2d.optimized_strongsort import OptimizedStrongSORT
from optimized_ingestion.stages.tracking_2d.tracking_2d import Tracking2D
from optimized_ingestion.stages.detection_3d.from_2d_and_road import From2DAndRoad
from optimized_ingestion.stages.tracking_3d.tracking_3d import Tracking3DResult
# from optimized_ingestion.trackers.yolov5_strongsort_osnet_tracker import TrackingResult
from optimized_ingestion.video import Video

In [ ]:
from optimized_ingestion.cache import disable_cache
disable_cache()

In [ ]:
BOSTON_VIDEOS = [
#     "scene-0757-CAM_FRONT",
    # "scene-0103-CAM_FRONT",
#     "scene-0553-CAM_FRONT",
    # "scene-0665-CAM_FRONT",
#     "scene-0655-CAM_FRONT",
#     "scene-0655-CAM_FRONT_RIGHT",
#     "scene-0655-CAM_BACK_RIGHT",
#     "scene-0553-CAM_FRONT_LEFT"
#     "scene-0103-CAM_FRONT"
]

NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"

In [ ]:
from optimized_ingestion.stages.detection_estimation.utils import compute_distance
pipeline = Pipeline()
pipeline.add_filter(filter=ParallelDecodeFrame())
pipeline.add_filter(filter=YoloDetection())

pipeline.add_filter(filter=From2DAndRoad())
pipeline.add_filter(filter=DetectionEstimation(
    lambda x: compute_distance(x.car_loc3d, x.ego_config.ego_translation) < 10))  # 5 Frame p Second
pipeline.add_filter(filter=StrongSORT())  # 2 Frame p Second

# pipeline.add_filter(filter=From2DAndRoad())
# pipeline.add_filter(filter=FilterCarFacingSideway())

if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos/boston-seaport", "frames.pickle"), "rb") as f:
    videos = pickle.load(f)

num_video = 0
for name, video in videos.items():
#     if name not in BOSTON_VIDEOS:
#         continue
#     if not name.endswith('CAM_FRONT'):
#         continue
    if 'CAM_FRONT' not in name:
        continue

    print(name, '--------------------------------------------------------------------------------')
    frames = Video(
        os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
        [camera_config(*f, 0) for f in video["frames"]],
        video["start"],
    )

    output = pipeline.run(Payload(frames))
    num_video += 1

total_runtime = 0
stage_runtimes = []
with_estimation_benchmarks = []
for stage in pipeline.stages:
    stage_runtimes.append({
        "stage": stage.classname(),
        "runtimes": stage.runtimes,
    })
    total_runtime += sum([run['runtime'] for run in stage.runtimes])

with_estimation_benchmarks.append({
    'stage_runtimes': stage_runtimes,
    'total_runtime': total_runtime
})
if num_video:
    with_estimation_benchmarks.append({'average runtime': sum([b['total_runtime'] for b in with_estimation_benchmarks])/num_video})


In [ ]:
with open("./outputs/fig14_with_optimization.txt", "w") as f3:
    json.dump(with_estimation_benchmarks, f3)

In [ ]:
without_estimation_pipeline = Pipeline()
without_estimation_pipeline.add_filter(filter=ParallelDecodeFrame())
without_estimation_pipeline.add_filter(filter=YoloDetection())

without_estimation_pipeline.add_filter(filter=From2DAndRoad())
without_estimation_pipeline.add_filter(filter=StrongSORT())  # 2 Frame p Second

# pipeline.add_filter(filter=From2DAndRoad())
# pipeline.add_filter(filter=FilterCarFacingSideway())

if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos/boston-seaport", "frames.pickle"), "rb") as f:
    videos = pickle.load(f)

num_video = 0
for name, video in videos.items():
#     if name not in BOSTON_VIDEOS:
#         continue
#     if not name.endswith('CAM_FRONT'):
#         continue
    if 'CAM_FRONT' not in name:
        continue

    print(name, '--------------------------------------------------------------------------------')
    frames = Video(
        os.path.join(DATA_DIR, "videos/boston-seaport", video["filename"]),
        [camera_config(*f, 0) for f in video["frames"]],
        video["start"],
    )

    output = without_estimation_pipeline.run(Payload(frames))
    num_video += 1

total_runtime = 0
stage_runtimes = []
without_estimation_benchmarks = []
for stage in without_estimation_pipeline.stages:
    stage_runtimes.append({
        "stage": stage.classname(),
        "runtimes": stage.runtimes,
    })
    total_runtime += sum([run['runtime'] for run in stage.runtimes])

without_estimation_benchmarks.append({
    'stage_runtimes': stage_runtimes,
    'total_runtime': total_runtime
})
if num_video:
    without_estimation_benchmarks.append({'average runtime': sum([b['total_runtime'] for b in without_estimation_benchmarks])/num_video})

In [ ]:
with open("./outputs/fig14_without_optimization.txt", "w") as f3:
    json.dump(without_estimation_benchmarks, f3)